In [2]:
import pandas as pd

In [16]:
df = pd.read_csv('../notion_data/원달러환율수익률.csv', encoding='cp949')

In [12]:
df.head()

,Date,Price,Close,High,Low,Open,Volume,?섏쑉?섏씡瑜?%)
0,2024-12-30,1472.800049,1475.250000,1466.359985,1472.800049,0.0,0.396053,NaN
1,2024-12-27,1466.989990,1487.040039,1462.800049,1466.989990,0.0,0.643520,NaN
2,2024-12-26,1457.609985,1470.290039,1454.770020,1457.609985,0.0,0.288282,NaN
3,2024-12-25,1453.420044,1458.079956,1448.449951,1453.420044,0.0,0.743058,NaN
4,2024-12-24,1442.699951,1459.920044,1449.329956,1442.699951,0.0,-0.188187,NaN


In [17]:
df = df[['Date', 'Close']]
df

,Date,Close
0,2024-12-30,1475.250000
1,2024-12-27,1487.040039
2,2024-12-26,1470.290039
3,2024-12-25,1458.079956
4,2024-12-24,1459.920044
...,...,...
2601,2015-01-06,1108.930054
2602,2015-01-05,1112.199951
2603,2015-01-02,1106.300049
2604,2015-01-01,1099.030029


In [18]:
df = df.rename(columns={'Date' : '날짜', 'Close' : '원자료'})

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2606 entries, 0 to 2605
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   날짜      2606 non-null   object 
 1   원자료     2605 non-null   float64
dtypes: float64(1), object(1)
memory usage: 40.8+ KB


In [20]:
df['날짜'] = pd.to_datetime(df['날짜'])

In [21]:
df = df.sort_values(by=['날짜'], ascending=True)

In [22]:
df = df.reset_index(drop=True)

df

,날짜,원자료
0,2014-12-31,NaN
1,2015-01-01,1099.030029
2,2015-01-02,1106.300049
3,2015-01-05,1112.199951
4,2015-01-06,1108.930054
...,...,...
2601,2024-12-24,1459.920044
2602,2024-12-25,1458.079956
2603,2024-12-26,1470.290039
2604,2024-12-27,1487.040039


In [36]:
# df = df.rename(columns/={'변환' : '날짜'})

In [23]:
import pandas as pd

# df['날짜'] = pd.to_datetime(df['날짜'])
df = df.sort_values('날짜').reset_index(drop=True)

# '날짜'를 인덱스로 설정
df = df.set_index('날짜')

# ,가 포함된 숫자 문자열을 정리하고 float으로 변환
df['원자료'] = df['원자료'].astype(str).str.replace(',', '').astype(float)

# 일별 수익률 계산
returns = df.pct_change()

# 결과 저장용 DataFrame 생성
result = pd.DataFrame(index=df.index)

# 앞으로 볼 윈도우 끝(일)과 라벨 정의: (5일→1~5, 10일→6~10, …)
windows = [
    (5, '(1~5)'),
    (10, '(6~10)'),
    (15, '(11~15)'),
    (20, '(16~20)')
]

# 각 컬럼에 대해 미래 5일씩 네 구간별 표준편차 계산
for col in df.columns:
    for end, label in windows:
        result[f"{col}{label}"] = (
            returns[col]
            .shift(-end)           # 미래 end일만큼 앞으로(음수) 시프트
            .rolling(window=5)     # 5일 윈도우
            .std()                 # 표준편차
        )

# 인덱스를 '날짜' 컬럼으로 되돌리고
result = result.reset_index()

print(result.head())


          날짜  원자료(1~5)  원자료(6~10)  원자료(11~15)  원자료(16~20)
0 2014-12-31       NaN        NaN         NaN         NaN
1 2015-01-01       NaN        NaN         NaN         NaN
2 2015-01-02       NaN        NaN         NaN         NaN
3 2015-01-05       NaN        NaN         NaN         NaN
4 2015-01-06  0.002703   0.004916    0.002313    0.004848


In [24]:
result.head(30)

,날짜,원자료(1~5),원자료(6~10),원자료(11~15),원자료(16~20)
0,2014-12-31,NaN,NaN,NaN,NaN
1,2015-01-01,NaN,NaN,NaN,NaN
2,2015-01-02,NaN,NaN,NaN,NaN
3,2015-01-05,NaN,NaN,NaN,NaN
4,2015-01-06,0.002703,0.004916,0.002313,0.004848
5,2015-01-07,0.003200,0.004899,0.002771,0.006733
6,2015-01-08,0.003834,0.005142,0.004654,0.005224
7,2015-01-09,0.004008,0.004183,0.004801,0.004070
8,2015-01-12,0.002685,0.004695,0.003839,0.004398
9,2015-01-13,0.004916,0.002313,0.004848,0.004324


In [25]:
result = result.rename(columns={
    '원자료(1~5)' : '원달러환율수익률표준편차(1~5)',
    '원자료(6~10)' : '원달러환율수익률표준편차(6~10)',
    '원자료(11~15)' : '원달러환율수익률표준편차(11~15)',
    '원자료(16~20)' : '원달러환율수익률표준편차(16~20)',    
})

In [26]:
result.head(30)

,날짜,원달러환율수익률표준편차(1~5),원달러환율수익률표준편차(6~10),원달러환율수익률표준편차(11~15),원달러환율수익률표준편차(16~20)
0,2014-12-31,NaN,NaN,NaN,NaN
1,2015-01-01,NaN,NaN,NaN,NaN
2,2015-01-02,NaN,NaN,NaN,NaN
3,2015-01-05,NaN,NaN,NaN,NaN
4,2015-01-06,0.002703,0.004916,0.002313,0.004848
5,2015-01-07,0.003200,0.004899,0.002771,0.006733
6,2015-01-08,0.003834,0.005142,0.004654,0.005224
7,2015-01-09,0.004008,0.004183,0.004801,0.004070
8,2015-01-12,0.002685,0.004695,0.003839,0.004398
9,2015-01-13,0.004916,0.002313,0.004848,0.004324


In [27]:
macro_df = pd.read_csv('../data/macro_data.csv')

In [28]:
macro_df.columns

Index(['회사명', 'stock_code', '상장일', 'kospi200(-20)', 'nasdaq(-20)',
       'putcall(-20)', 'VIXCLS', '미국10년물국채금리수익률(-20)', '시장금리국고채3년',
       '환율변동률(-20)',
       ...
       '코스피수익률표준편차(11~15)', '코스피수익률표준편차(16~20)', '나스닥수익률표준편차(1~5)',
       '나스닥수익률표준편차(6~10)', '나스닥수익률표준편차(11~15)', '나스닥수익률표준편차(16~20)',
       'S&P500수익률표준편차(1~5)', 'S&P500수익률표준편차(6~10)', 'S&P500수익률표준편차(11~15)',
       'S&P500수익률표준편차(16~20)'],
      dtype='object', length=172)

In [29]:
# 1) Convert both columns to datetime
macro_df['상장일'] = pd.to_datetime(macro_df['상장일'])
result['날짜'] = pd.to_datetime(result['날짜'])

# 2) Sort both DataFrames by their date keys
merged_full = macro_df.sort_values('상장일').reset_index(drop=True)
result = result.sort_values('날짜').reset_index(drop=True)

merged_full = pd.merge_asof(
    merged_full,
    result,
    left_on='상장일',
    right_on='날짜',
    direction='backward'   # 상장일 당일 또는 그 이전의 가장 가까운 거래일
).drop(columns=['날짜'])

In [30]:
merged_full.columns

Index(['회사명', 'stock_code', '상장일', 'kospi200(-20)', 'nasdaq(-20)',
       'putcall(-20)', 'VIXCLS', '미국10년물국채금리수익률(-20)', '시장금리국고채3년',
       '환율변동률(-20)',
       ...
       '나스닥수익률표준편차(11~15)', '나스닥수익률표준편차(16~20)', 'S&P500수익률표준편차(1~5)',
       'S&P500수익률표준편차(6~10)', 'S&P500수익률표준편차(11~15)', 'S&P500수익률표준편차(16~20)',
       '원달러환율수익률표준편차(1~5)', '원달러환율수익률표준편차(6~10)', '원달러환율수익률표준편차(11~15)',
       '원달러환율수익률표준편차(16~20)'],
      dtype='object', length=176)

In [31]:
merged_full.to_csv('macro_data.csv', encoding='utf-8-sig', index=False)
